In [1]:
import numpy as np
import pandas as pd


In [3]:
sleep_df=pd.read_csv('sleepdata_2.csv',delimiter=";")
sleep_df.sample(n=5)
sleep_df

0                   NaN
1                   NaN
2                   NaN
3                   NaN
4                   NaN
             ...       
916    Central Kootenay
917      North Okanagan
918      North Okanagan
919    Central Kootenay
920    Central Kootenay
Name: City, Length: 921, dtype: object

In [4]:
n_rows=sleep_df.shape[0]
n_cols=sleep_df.shape[1]
n_rows, n_cols

(921, 21)

#TODO: GIVE OBSERVATION ABOUT MEANING OF EACH ROWS, COLUMNS

- After finding meaning of each rows, each columns in the data, we check whether data have duplicate rows

In [5]:
duplicated_rows = sleep_df[sleep_df.duplicated()]
len(duplicated_rows)


0

There is no duplicate row in data. We notice that there are many missing data (NaN) in the CSV file. We need to find how many percentage of missing data there are.

In [6]:
sleep_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 921 entries, 0 to 920
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Start                        921 non-null    object 
 1   End                          921 non-null    object 
 2   Sleep Quality                921 non-null    object 
 3   Regularity                   921 non-null    object 
 4   Mood                         0 non-null      float64
 5   Heart rate (bpm)             921 non-null    int64  
 6   Steps                        921 non-null    int64  
 7   Alarm mode                   921 non-null    object 
 8   Air Pressure (Pa)            492 non-null    float64
 9   City                         487 non-null    object 
 10  Movements per hour           921 non-null    float64
 11  Time in bed (seconds)        921 non-null    float64
 12  Time asleep (seconds)        921 non-null    float64
 13  Time before sleep (s

Now we will caculate missing percent of each column

In [7]:
missing_values = sleep_df.isnull().sum()

# Calculate missing ratio for each column
missing_ratio = (missing_values / len(sleep_df)) * 100
missing_ratio

Start                            0.000000
End                              0.000000
Sleep Quality                    0.000000
Regularity                       0.000000
Mood                           100.000000
Heart rate (bpm)                 0.000000
Steps                            0.000000
Alarm mode                       0.000000
Air Pressure (Pa)               46.579805
City                            47.122693
Movements per hour               0.000000
Time in bed (seconds)            0.000000
Time asleep (seconds)            0.000000
Time before sleep (seconds)      0.000000
Window start                     8.686211
Window stop                      8.686211
Did snore                        0.000000
Snore time                       0.000000
Weather temperature (°C)         0.000000
Weather type                     0.000000
Notes                           99.022801
dtype: float64

From observing data and the result, we will drop columns that have >40% data missing, 

In [8]:
columns_to_drop = ['Mood','Air Pressure (Pa)','City','Notes']
sleep_df=sleep_df.drop(columns_to_drop,axis=1)

Check if any columns have same value in every cells and drop that columns

In [9]:
for column in sleep_df.columns:
    if sleep_df[column].nunique() == 1:
        print("Are all values in '{}' the same".format(column))
        sleep_df=sleep_df.drop(column,axis=1)

Are all values in 'Heart rate (bpm)' the same


## CATEGORICAL COLUMNS

Convert columns 'Start', 'End', 'Window start', 'Window stop' to datetime

In [10]:
sleep_df['Start']=pd.to_datetime(sleep_df['Start'],format='%Y-%m-%d %H:%M:%S')
sleep_df['End']=pd.to_datetime(sleep_df['End'],format='%Y-%m-%d %H:%M:%S')
sleep_df['Window start']=pd.to_datetime(sleep_df['Window start'],format='%Y-%m-%d %H:%M:%S')
sleep_df['Window stop']=pd.to_datetime(sleep_df['Window stop'],format='%Y-%m-%d %H:%M:%S')


COUNT UNIQUE VALUE OF columns `Alarm mode`, `Did snore` and `Weather type`  

In [11]:
columns_to_count=['Alarm mode','Did snore','Weather type']
unique_values_dict = {column: sleep_df[column].unique().tolist() for column in columns_to_count}
unique_values_dict

{'Alarm mode': ['Normal', 'No alarm'],
 'Did snore': [True, False],
 'Weather type': ['No weather',
  'Sunny',
  'Fair',
  'Partly cloudy',
  'Cloudy',
  'Snow',
  'Fog',
  'Rain',
  'Rainy showers']}

Value of `Alarm mode`, `Did snore` and `Weather` type seem normal

## NUMERIC COLUMNS

We see that columns `Sleep Quality` and `Regularity` contains percentage values. I think that we should change it to numeric format. Currently, they are stored as object type

In [12]:
sleep_df['Sleep Quality'] = sleep_df['Sleep Quality'].str.rstrip('%').astype('float')
sleep_df.rename(columns={'Sleep Quality': 'Sleep Quality (%)'}, inplace=True)

sleep_df['Regularity'] = sleep_df['Regularity'].str.rstrip('%').astype('float') 
sleep_df.rename(columns={'Regularity': 'Regularity (%)'}, inplace=True)

In [13]:
sleep_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 921 entries, 0 to 920
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Start                        921 non-null    datetime64[ns]
 1   End                          921 non-null    datetime64[ns]
 2   Sleep Quality (%)            921 non-null    float64       
 3   Regularity (%)               921 non-null    float64       
 4   Steps                        921 non-null    int64         
 5   Alarm mode                   921 non-null    object        
 6   Movements per hour           921 non-null    float64       
 7   Time in bed (seconds)        921 non-null    float64       
 8   Time asleep (seconds)        921 non-null    float64       
 9   Time before sleep (seconds)  921 non-null    float64       
 10  Window start                 841 non-null    datetime64[ns]
 11  Window stop                  841 non-null    

Caculate quantile of numeric columns

In [14]:
numeric_cols=['Sleep Quality (%)', 'Regularity (%)', 'Steps', 'Movements per hour', 'Time in bed (seconds)', 'Time asleep (seconds)', 'Time before sleep (seconds)', 'Snore time', 'Weather temperature (°C)']
numeric_col_profile = sleep_df[numeric_cols].quantile([0, 0.25, 0.5, 0.75, 1])
numeric_col_profile.index = ['min', '25%', '50%', '75%', 'max']
numeric_col_profile

,Sleep Quality (%),Regularity (%),Steps,Movements per hour,Time in bed (seconds),Time asleep (seconds),Time before sleep (seconds),Snore time,Weather temperature (°C)
min,7.0,-1.0,0.0,0.0,1852.8,0.0,0.0,0.0,-19.0
25%,71.0,82.0,2299.0,37.0,25500.6,21627.5,449.9,0.0,0.0
50%,78.0,86.0,4245.0,49.6,27422.9,23919.0,563.7,148.0,0.0
75%,86.0,91.0,7452.0,65.2,29530.1,26257.0,1287.0,440.0,3.1
max,100.0,100.0,38165.0,17926.7,46703.4,45769.4,5677.7,4477.3,24.5


In [16]:
sleep_df.to_csv('Processed.csv')